In [1]:
import json
import numpy as np
import gurobipy
import utils
from gurobipy import Model, GRB, quicksum
import pandas as pd
import networkx as nx
import plotly.express as px

from utils import create_planning, create_tables, dominate, ModelData, ModelPreordreData, get_preorder, create_graph, get_non_dominated

In [2]:
m, data = utils.create_model('instances/toy_instance.json')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12


In [3]:
filtered_solutions, scenario = get_non_dominated(m, data)

Starting optimzation...
done.


In [4]:
df_solutions = pd.DataFrame(-1*np.array(filtered_solutions))
#df_solutions['scenario'] = scenario
df_solutions.rename(columns={0: 'f1', 1: 'f2', 2:'f3'}, inplace=True)

In [5]:
reference_cars = np.array([ [173, 11.4, 10.01, 10, 7.88, 49500],
                            [176, 12.3, 10.48, 11, 7.96, 46700],
                            [142, 8.2, 7.3, 5, 5.65, 32100],
                            [148, 10.5, 9.61, 7, 6.15, 39150],
                            [178, 14.5, 11.05, 13, 8.06, 64700],
                            [180, 13.6, 10.4, 13, 8.47, 75700],
                            [182, 12.7, 12.26, 11, 7.81, 68593],
                            [145, 14.3, 12.95, 11, 8.38, 55000],
                            [161, 8.6, 8.42, 7, 5.11, 35200],
                            [117, 7.2, 6.75, 3, 5.81, 24800] ])

In [6]:
ranking = np.arange(4)
choices = df_solutions.copy().to_numpy()

In [7]:
def generate_pairs(ranking: list):
    pairs = []
    for i in range(len(ranking)):
        for j in range(i+1, len(ranking)):
            pairs.append((ranking[i], ranking[j]))
    return pairs
pairs = generate_pairs(ranking)

In [46]:
pairs = [
    (1, 3),
]

In [104]:
model_uta = Model("UTA")

nbRanking = len(ranking)
dim = choices.shape[1]
L = 3
dim_vals = []
for i in range(dim):
    dim_vals.append(np.linspace(choices[:, i].min(), choices[:, i].max(), L))

s_i_x_k = {}
for i in range(dim): 
    s_i_x_k[i] = {}
    for k in range(L):
        s_i_x_k[i][k] = model_uta.addVar(vtype=GRB.CONTINUOUS, name=f's_{i}(x_{k})')

s = {}
for idx in range(nbRanking):
    point = ranking[idx]
    sum_dim = 0
    for d in range(dim):
        multiplier = (choices[point][d] - dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()]) / (dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()+ 1 ] - dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()])
        sum_dim += s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()] + multiplier*(s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()+1] - s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()])
    s[point] = sum_dim

eps_pos = {ranking[j]: model_uta.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f'sig_pos_{j}') for j in range(nbRanking)}
eps_neg = {ranking[j]: model_uta.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f'sig_neg_{j}') for j in range(nbRanking)}
epsilon = 0.01

for i,j in pairs:
        model_uta.addConstr(s[i] - eps_pos[i] + eps_neg[i] >= s[j] - eps_pos[j] + eps_neg[j] + epsilon)

for i in range(dim):
    for k in range(L-1):
        model_uta.addConstr(s_i_x_k[i][k+1] - s_i_x_k[i][k] >= epsilon) 

for i in range(dim):
    model_uta.addConstr(s_i_x_k[i][0] == 0)

model_uta.addConstr(quicksum([s_i_x_k[i][L-1] for i in range(dim)]) == 1)

f1 = quicksum(eps_pos.values()) + quicksum(eps_neg.values())

model_uta.setObjective(f1, GRB.MINIMIZE)
model_uta.update()

In [105]:
model_uta.params.outputflag = 0
model_uta.reset()
model_uta.optimize()

In [115]:
scores = []
for point in range(len(choices)):
    sum_dim = 0
    for d in range(dim):
        multiplier = (choices[point][d] - dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()]) / (dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()+ 1 ] - dim_vals[d][(dim_vals[d] - choices[point][d]).argmin()])
        sum_dim += s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()] + multiplier*(s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()+1] - s_i_x_k[d][(dim_vals[d] - choices[point][d]).argmin()])

    scores.append(sum_dim.getValue())

In [117]:
df_solutions['scores'] = scores

In [120]:
df_solutions
df_solutions.sort_values(by='scores', ascending=False)

,f1,f2,f3,scores
0,0,0,0,1.920000
8,10,-1,-2,1.740350
15,20,-1,-3,1.738427
1,10,-2,-1,1.572622
9,20,-2,-2,1.570699
16,32,-2,-3,1.569392
2,20,-3,-1,1.402972
10,30,-3,-2,1.401049
17,42,-3,-3,1.399741
11,40,-4,-2,1.231399


### weighted

In [5]:
preference_list = [
    (7,   9),
    (10, 15),
    (7,   1)
]

choices = filtered_solutions.copy()
preordre_model, preordre_data = get_preorder(choices, preference_list)

G = create_graph(preordre_data)
df_solutions['degree'] = [val for idx, val in G.degree]
df_solutions.sort_values(by='degree', ascending=False, inplace=True)

df_solutions

,f1,f2,f3,scenario,degree
7,59,-11,-1,27,20
6,49,-9,-1,25,19
5,47,-8,-1,24,17
3,30,-5,-1,21,15
4,37,-6,-1,22,14
12,50,-5,-2,37,14
17,42,-3,-3,51,14
11,40,-4,-2,36,13
10,30,-3,-2,35,13
16,32,-2,-3,50,12


In [8]:
fig = px.scatter_3d(df_solutions, x='f2', y='f3', z='f1', color='degree')
fig.update_layout(
    autosize=False,
    width=1000,
    height=400,
    )

fig.update_traces(marker={'size': 5})
fig.show()

In [7]:
m.params.ScenarioNumber = df_solutions.iloc[0]['scenario']
tab_a, tab_b = create_tables(data)
display(tab_a)
display(tab_b)
create_planning(data)

,A,B,C
membre,,,
Olivia,1,1,1
Liam,1,1,0
Emma,0,0,1


,A,B,C
projet,,,
Job1,1,1,1
Job2,1,2,0
Job3,1,0,2
Job4,0,2,1
Job5,0,0,2


,0,1,2,3,4
membre,,,,,
Olivia,C_Job5,X,A_Job1,C_Job3,B_Job4
Liam,X,X,X,X,B_Job4
Emma,C_Job5,X,C_Job1,C_Job3,C_Job4
